# Multidomain

This notebook is an example of how to create a multidomain topology.

## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")
    

## (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:    
    output = fablib.list_sites()
except Exception as e:
    print(f"Exception: {e}")

## Create the Experiment Slice

The following creates private layer-2 network and the nodes


In [ ]:
from datetime import datetime
from dateutil import tz

local_network_base_name='net_local'

node_names=["sdx-c", "lc-1", "lc-2", "lc-3", "mininet", "monitor"]
node_ips=["192.168.201.204", "192.168.201.205", "192.168.201.206", "192.168.201.207", "192.168.201.208", "192.168.201.203"]

site1_local_net_name = f'{local_network_base_name}1'
 
###worker1=f'{site1.lower()}-w1.fabric-testbed.net'
###worker2=f'{site1.lower()}-w2.fabric-testbed.net'
    
cores=4
ram=16
disk=100
image = 'docker_rocky_8'

In [ ]:
[site1,site2,site3] = fablib.get_random_sites(count=3,avoid=['SRI','LOSA','NEWY'])
print(f"Sites: {site1},{site2},{site3}")

site1="FIU"

slice_name=f"Slice AWSDX Controllers {site1} {datetime.now()}"
print (f"Slice Name: {slice_name}")

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    #Create Site1 Nodes
    site1_local_ifaces = [ ]
    for i in range(len(node_names)):
        ###node = slice.add_node(name=node_names[i], site=site1, host=worker1, cores=cores, ram=ram, disk=disk, image=image)
        node = slice.add_node(name=node_names[i], site=site1, cores=cores, ram=ram, disk=disk, image=image)
        site1_local_ifaces.append(node.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0])
      
    #Create Site Local Networks
    site1_local_net = slice.add_l2network(name=site1_local_net_name, interfaces=site1_local_ifaces)

    #Submit Slice Request
    slice_id = slice.submit()

except Exception as e:
    print(f"Slice Fail: {e}")

## Observe the Slice's Attributes


In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

## Configure IP Addresses

The following cells show how to pick IPv4 subnets for each network and assign addresses from those subnets to the appropriate interfaces on the nodes.


### Pick Subnets

Pick the subnet for the local networks.  The /24 subnets can support up to 254 locally connected nodes plus the gateway. 


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    # Local Subnets
    net_local1_subnet = IPv4Network("192.168.201.0/24")
    net_local1_available_ips = list(net_local1_subnet)[1:]
    net_local1_gateway = net_local1_available_ips.pop(0)
       
    print(f"Site1: subnet: {net_local1_subnet}, gateway: {net_local1_gateway}")

except Exception as e:
    print(f"Exception: {e}")

### Configure Local Node IPs

Configure the local nodes with addresses from the local subnet available address list. 

Add the router link subnets if you want to access the routers (i.e. if you want to `ping` the routers or find paths with `tracepath`)

Collect a list of local dataplane IPs to target for testing

In [ ]:
local_dataplane_ips = {}

In [ ]:
try:    
    #Create Site1 Nodes
    for i in range(len(node_names)):
        name=node_names[i]
        node = slice.get_node(name=name)
        
        ###node_addr = net_local1_available_ips.pop(0)
        node_addr = node_ips[i]
        
        node_iface = node.get_interface(network_name=site1_local_net_name)  
        node_iface.ip_addr_add(addr=node_addr, subnet=net_local1_subnet)
                
        #Collect dataplane IP for testing
        local_dataplane_ips[name] = node_addr
        
        print(f"Node {name} dataplane IP: {node_addr}")    
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        stdout, stderr = node.execute(f'sudo sed -i "s/SELINUX=enforcing/SELINUX=disabled/g" /etc/selinux/config')
        stdout, stderr = node.execute(f'sudo setenforce 0')
        stdout, stderr = node.execute(f'sestatus')


except Exception as e:
    print(f"Exception: {e}")
    

In [ ]:
try:
    source_node_name =  "sdx-c"
   
    source_node = slice.get_node(name=source_node_name)
    for node_name,target_ip in local_dataplane_ips.items():
        print(f"Testing target node: {node_name}, target IP: {target_ip}")
    
        stdout, stderr = node.execute(f'ping -c 5 {target_ip}')

        stdout, stderr = node.execute(f'tracepath {target_ip}')
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    slice.save("awsdx_1.graphml")
except Exception as e:
    print(f"Exception: {e}")

## Get and Renew the Slice

You slice is in the list of all your slices. You can loop through the list of slices to get the slice. Python has a standard tool to filter lists. Try using a lambda function to filter out your slice using its name.

The new end date is in UTC.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

#Set end host to now plus 1 day
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

### Check New Lease End Date


In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End (UTC)        : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")

### Configure VMs wrt Automation and Ansible Control Node

This complex configuration is handled through a bash script `awsdx_node_configure.sh` that resides in the folder containing this notebook. The script is executed by, first, uploading the script with the `node.upload_file()` FABLib method.  Then the script is executed using the `node.execute()` FABLib method.  Note that the script passes the OS interfaces names and configured IPs as arguments from the notebook to the script.  

These scripts take a while to run. You may wish to use a separate terminal window to ssh to the VMs and tail the log file to watch the progress with: `tail -F awsdx_node_configure.log`.

In [ ]:
try: 
    #thread1 = execute.thread()
    #stdout, stderr = thread1.result()
    
    #slice_name = "Slice AWSDX Controllers 2023-05-14 20:47:11.813885"
    slice = fablib.get_slice(name=slice_name)

    for i in range(len(node_names)):
        name=node_names[i]
        node = slice.get_node(name=name)
        node.upload_file('./config/awsdx_node_configure.sh','awsdx_node_configure.sh')
        node.upload_file('./config/awsdx_ansible_configure.sh','awsdx_ansible_configure.sh')        
        node.upload_file('./config/id_rsa','id_rsa')        
        node.upload_file('./config/id_rsa.pub','id_rsa.pub')        
        node_config_thread = node.execute_thread(f'chmod +x awsdx_node_configure.sh && ./awsdx_node_configure.sh')

except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()            
     

In [ ]:
try:
    
    monitor = slice.get_node(name="monitor")
    monitor.upload_file('./config/awsdx_ansible_configure.sh','/var/tmp/awsdx_ansible_configure.sh')        
    monitor_ansible_thread = monitor.execute_thread(f'chmod +x /var/tmp/awsdx_ansible_configure.sh && sudo chown awsdx-admin. /var/tmp/awsdx_ansible_configure.sh')

    #Join Threads
    #print(f"Joining Threads")
    #stdout, stderr = monitor_ansible_thread.result() 
    #print(f"Monitor: ", stdout, stderr)
    
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()            

